In [ ]:
"""Run two below lines to get my_abc module"""
# !git clone https://github.com/thanhttttt/thanh.git
# !pip install -r /content/thanh/requirements.txt
"""Run two below lines to drive"""
# from google.colab import drive
# drive.mount('/content/drive')

# Import lib

In [ ]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import Sionna
import sys
sys.path.append('../')
sys.path.append('/content/thanh/')
sys.path.append('../thanh/')
import sionna

# try:
#     import sionna
# except ImportError as e:
#     # Install Sionna if package is not already installed
#     import os
#     os.system("pip install sionna")
#     import sionna

import tensorflow as tf
# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.config.seed = 42 # Set seed for reproducible results

# Load the required Sionna components
from sionna.nr.my_abc import *

# Read parquet and load data

In [6]:
dataset_dir = f'../Pusch_data/dataset'
pickles_dir = f'{dataset_dir}/pickle'
hdf5_dir = f'{dataset_dir}/hdf5'
parquet_dir = f'{dataset_dir}/parquet'

In [7]:
parquet_name = '20250312043946895638'
df = pd.read_parquet(f'{parquet_dir}/{parquet_name}.parquet', engine="pyarrow").reset_index()
df

,index,nPhyCellId,nSFN,nSlot,nPDU,nGroup,nUlsch,nUlcch,nRachPresent,nRNTI,...,nTPPuschID,nRxRUIdx,nUE,nPduIdx,Channel_model,Speed,Delay_spread,Esno_db,Data_filename,Data_dirname
0,0,442,0,4,1,1,1,0,0,40035,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],CDL-A,10.0,10.0,-5.0,20250312043951342532,20250312043946895638
1,1,228,0,4,1,1,1,0,0,25203,...,0,"[0, 1, 2, 3, 4, 5, 6, 7]",1,[0],CDL-C,4.0,50.0,-5.0,20250312043956263574,20250312043946895638


In [1]:
# Load train_data into Dataset
BATCH_SIZE = 2
train_set = tf.data.Dataset.from_generator(lambda: data_loader(df, hdf5_dir),
                        output_types=(tf.int32, tf.float32, tf.float32, tf.complex64, tf.complex64))

train_set = train_set.cache()
train_set = train_set.prefetch(tf.data.AUTOTUNE)
train_set = train_set.map(preprocessing).batch(BATCH_SIZE)

for n, (index, esno_db, c, y, r) in enumerate(train_set):
    print(y.shape, esno_db)
    y_r = tf.concat([y, r], axis = -1) # Model input

NameError: name 'tf' is not defined

# Read config from dataframe

In [16]:
puschCfg = pusch_config_from_pd_row(df.iloc[0])
puschCfg.show()

Carrier Configuration
cyclic_prefix : normal
cyclic_prefix_length : 2.3437500000000002e-06
frame_duration : 0.01
frame_number : 0
kappa : 64.0
mu : 1
n_cell_id : 442
n_size_grid : 162
n_start_grid : 0
num_slots_per_frame : 20
num_slots_per_subframe : 2
num_symbols_per_slot : 14
slot_number : 4
sub_frame_duration : 0.001
subcarrier_spacing : 30
t_c : 5.086263020833334e-10
t_s : 3.2552083333333335e-08

PUSCH Configuration
My_Config : MyConfig(Sys=SystemConfig(NCellId=442, FrequencyRange=1, BandWidth=60, Numerology=1, CpType=0, NTxAnt=1, NRxAnt=8, BwpNRb=162, BwpRbOffset=0, harqProcFlag=0, nHarqProc=1, rvSeq=0), Ue=[UeConfig(TransformPrecoding=0, Rnti=40035, nId=442, CodeBookBased=0, DmrsPortSetIdx=[0], NLayers=1, NumDmrsCdmGroupsWithoutData=2, Tpmi=0, FirstSymb=0, NPuschSymbAll=14, RaType=1, FirstPrb=0, NPrb=130, FrequencyHoppingMode=0, McsTable=0, Mcs=8, ILbrm=0, nScId=0, NnScIdId=442, DmrsConfigurationType=0, DmrsDuration=1, DmrsAdditionalPosition=1, PuschMappingType=0, DmrsTypeAPositi

In [13]:
get_unique_configs(df)

,nPhyCellId,nCpType,nSubcSpacing,nBWPSize,nBWPStart,nSlot,nDMRSConfigType,nNrOfDMRSSymbols,nDMRSAddPos,nPortIndex,...,nStartSymbolIndex,nNrOfSymbols,nRBStart,nRBSize,nPTRSPresent,nAck,nCsiPart1,nCsiPart2,nTpPi2BPSK,indices
0,228,0,1,162,0,4,0,1,1,"(0,)",...,0,14,16,130,0,0,0,0,0,[1]
1,442,0,1,162,0,4,0,1,1,"(0,)",...,0,14,0,130,0,0,0,0,0,[0]
